# Introduction

Il s'agit d'un retour sur les premiers développements de VApp, une application qui tente de répondre, à l'aide de la GenAI, au besoin d'identifier rapidement et sans grandes compétences techniques l'éligibilité des aides pour un projet.

# Raison de certains choix technique

Le code associé à chaque partie sera détaillé plus bas dans le document Markdown/Jupyter.

Nous nous appuyons principalement sur l'API d'Aide Territoire pour constituer la base de données d'maide. Cette API représente en effet une première source fiable et régulièrement mise à jour concernant les subventions pour les collectivités.

Nous avons également évoqué la possibilité d'interroger les formulaires d'aide de Démarches Simplifiées, mais cela présente un défi de taille. En effet, toutes les aides ne sont pas centralisées sur Démarches Simplifiées.

J'ai imaginé l'application autour de trois composantes principales :
- Frontend
- Backend
- Serveur GPU dédié à la partie LLM et à l'IA de manière plus générale
Je vais détailler le choix de chacune de ces solutions par la suite. Pour résumé :

Dans le cadre de la création d'une application web basée sur l'intelligence artificielle, j'ai choisi de structurer l'architecture de manière modulaire en séparant clairement le frontend, le backend et un serveur GPU dédié aux tâches liées à l'IA. L'objectif principal derrière cette approche est d'optimiser les performances et de garantir une meilleure gestion des ressources.

En effet, les GPU sont essentiels pour accélérer les calculs massivement parallèles, caractéristiques des modèles d'apprentissage automatique et des réseaux neuronaux complexes. Ces derniers nécessitent de traiter de grandes quantités de données en simultané, ce qui rend les GPU bien plus efficaces que les CPU pour ces types de tâches. En déployant un serveur spécifique aux calculs IA, je m'assure que l'inférence et l'entraînement des modèles de type LLM se déroulent de manière fluide, sans ralentir le reste de l'application.

Ce découplage permet également d'améliorer la scalabilité. Si les besoins en calcul augmentent, il est possible d'ajouter davantage de ressources GPU sans impacter la performance du frontend ou du backend. Cela réduit les risques de goulots d'étranglement et offre une meilleure flexibilité en termes d'évolutivité. Par ailleurs, cette architecture permet de mieux contrôler les coûts en allouant des ressources puissantes là où elles sont vraiment nécessaires, tout en maintenant une infrastructure classique pour les autres composants de l'application​.

En conclusion, cette séparation claire entre les différentes couches de l'application et l'utilisation d'un serveur dédié aux calculs IA garantit non seulement des performances optimales, mais aussi une flexibilité et une évolutivité accrues, essentielles pour l'évolution future du projet.

## Frontend
Le choix du frontend a été relativement simple. Je travaillais initialement avec Python Flask en backend, et j'utilisais directement le combo HTML/CSS/JS. Ensuite, j'ai voulu implémenter des fonctionnalités plus complexes et plus dynamiques, et j'ai hésité entre Angular et React.

La communauté beta.gouv utilisant principalement React et proposant déjà un template React DFSR, mon choix s'est orienté vers cette solution.

## Backend
Initialement sur Flask, je suis rapidement passé à Django. Django est un framework Python avec une communauté déjà importante chez beta.gouv. De plus, il semblait plus adapté pour créer un backend sous forme d'API, ce qui permettrait une intégration plus simple à long terme avec Aide Territoire ou Mon Espace Collectivité, selon moi.

Je souhaitais également rester sur un framework Python, car cela simplifie grandement les interactions avec les LLM, quelle que soit l'approche adoptée par la suite.

## Serveur GPU - IA
Pour le serveur GPU IA j'ai préféré partir sur une solution très low level, pour plusieurs raison :
- Des application comme ollama permette une interaction et un déploiement extraimeent simple sous forme d'API à un large panel de LLM
- Permet de facilement avoir accès à de la ressoure GPU qui est reconnue pour faciliter l'usage de large réseau neuronaux.
- Pour le moment le problème va être brute force à l'aide de LLM mais il sera envisagble de a termes créer des réseau de neurone dédié à la tache beaucoup plus éfficace et ne nécessitant pas de autant de ressoure. Le serveur dédié facilitera le protoypage de ces solutions.
- Dans notre contexte nous aurions pu utiliser des api tel que celle d'open IA ou de mistral, cependant nous voulons pouvoir tester rapidement une large variété de model open source pour trouver celui le plus adapter à la tache, quit à employer également une compinéaison d'autre méthode. Nous avons également pour ojbectif de montrer la faisablité d'utiliser des modèles customes sur des serveur spécialisé pour mettre en évidence où non la néssésité à plus long terme de se dauter de serveur dédié GenIA au seins des ministères.

### Usage de la solution Ollama
Je souhaite utiliser Ollama (https://ollama.com/) pour la gestion des LLM, car il permet une gestion extrêmement simplifiée des modèles via une API, tout en offrant la possibilité d'ajuster très finement les modèles pour des utilisations spécialisées.

Il intègre également plusieurs méthodes d'optimisation, notamment la quantization, ce qui permet de réduire considérablement l'utilisation des ressources serveur.

Enfin, je pense qu'il n'est pas nécessaire de nous concentrer sur l'optimisation ultime des appels aux LLM pour le moment, car cela pourra être pris en compte plus tard dans une application à plus grande échelle, produite par l'État. De plus, nous pouvons facilement faire évoluer les instances d'Ollama directement via le backend de notre application (par exemple, en prévoyant une gestion de la charge qui ouvre ou ferme des instances Ollama en fonction des besoins).

# Collecte des données d'aides territoires

In [ ]:
# Import des libraires
import requests
import pandas as pd
import numpy as np
import json
import os

In [ ]:
# Creation d'une classe pour dialoguer plus simplement avec l'API de AT
class APIClient:
    def __init__(self,x_auth_token):
        self.x_auth_token = x_auth_token
        self.base_uri = self.get_api_base_url()
        self.bearer_token = self.get_bearer_token()
        self.session = self.create_session() 

    def get_api_base_url(self):
        return 'https://aides-territoires.beta.gouv.fr/api/'

    def get_bearer_token(self):
        url = 'https://aides-territoires.beta.gouv.fr/api/connexion/'
        headers = {
            'accept': 'application/json',
            'Content-Type': 'application/json',
            'X-AUTH-TOKEN': self.x_auth_token
        }
        response = requests.post(url, headers=headers)
        response.raise_for_status()
        return response.json()['token']

    def create_session(self):
        session = requests.Session()
        session.headers.update({
            'Authorization': f'Bearer {self.bearer_token}',
            'Accept': 'application/json'
        })
        return session

    def simple_request_endpoint(self, endpoint, method='GET'):
        url = f'{self.base_uri}{endpoint}'
        response = self.session.request(method, url)
        response.raise_for_status()
        return response

    def simple_request_url(self, url, method='GET'):
        response = self.session.request(method, url)
        response.raise_for_status()
        return response
    
# Utilisation vous avez besoin d'un x_auth_token fournit par aide territoires
client = APIClient(x_auth_token)

Je collecte l'ensemble des données de toutes les aides d'Aide Territoire (AT) que je stocke dans une liste, afin de pouvoir ensuite les convertir en Pandas DataFrame et les enregistrer en local. Je procède ainsi pour plusieurs raisons (tout en sachant pertinemment que l'idéal serait d'avoir notre propre base de données) :

- La base de données n'est pas très volumineuse, environ 50 Mo tout au plus.
- Le LLM représente la plus grande part du temps de réponse. Cependant, durant la phase de prototypage, nous préférons éviter les problèmes de latence et l'instabilité potentielle liée à l'API d'AT.
- Stocker les données dans un Pandas DataFrame permet de visualiser rapidement et de manière exhaustive le contenu de la base de données.

In [ ]:
response = client.simple_request_endpoint('aids/?page=1&itemsPerPage=30')
data = response.json()
data_collect = data['results']
error_counter = 0
while data['next'] or error_counter >10:
    try:
        url = data['next']
        print(url)
        response = client.simple_request_url(url)
        data = response.json()
        data_collect = data_collect + data['results']
        error_counter = 0
    except Exception as error:
        print(error)
        error_counter += 1

data_at = pd.DataFrame(data_collect)

Pour l'utilisation de la base de données, nous avons remarqué qu'un grand nombre d'aides contiennent moins de 3 000 caractères (environ 500 mots). Cela représente 90 % de la base, mais pour garantir l'efficacité de l'outil, nous avons préféré les exclure et ne conserver que les aides les mieux décrites, soit un total de 400.

In [ ]:
data_at_select = data_at[['id','name','description','eligibility','project_examples']].dropna(subset='description')

sub_min_description = 3000

data_at_select = data_at_select[data_at_select['description'].apply(len)>sub_min_description].reset_index(drop=True)

# Usage du LLM

## Usage 1 - Attribution d'un score en fonction d'une aide et de la description d'un projet


Ici, nous sommes grandement contraints par les limites techniques propres aux LLM. Voici les principales raisons de mon approche :

- Les aides contiennent entre 500 et 10 000 mots, soit environ 250 à 5 000 tokens. Les LLM ayant une capacité limitée à gérer le contexte (et plus le contexte est long, moins ils sont fiables), j'ai préféré limiter chaque appel à une seule aide et à une description de projet.
- Cette approche est similaire à d'autres méthodes déjà connues avec les LLM.
- Le RAG (Retrieval-Augmented Generation) a été écarté pour : UNE VRAIE RAISON À AJOUTER.
- Nous bouclons sur les réponses car... EXPLICATION À AJOUTER.

In [ ]:
LE CODE à ajouter

## Usage 2 - Questionner sur une aide pour en améliorer l'éligibilité

In [ ]:
EN CONSTRUCTION